In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
# Load the data
data = pd.read_csv('AQI.csv')
data['time'] = pd.to_datetime(data['time'])
data.set_index('time')
time = data['time']

fold = 3
# Split into training, validation, and test sets according to time
# fold 1
if fold == 1:
    train = data[(time >= '2018-01-01 00:00:00') & (time <= '2018-12-31 23:00:00')]
    validation = data[(time >= '2019-01-01 00:00:00') & (time <= '2019-12-31 23:00:00')]
# fold 2
if fold == 2:
    train = data[(time >= '2018-01-01 00:00:00') & (time <= '2019-12-31 23:00:00')]
    validation = data[(time >= '2020-01-01 00:00:00') & (time <= '2020-12-31 23:00:00')]
# fold 3
if fold == 3:
    train = data[(time >= '2018-01-01 00:00:00') & (time <= '2020-12-31 23:00:00')]
    validation = data[(time >= '2021-01-01 00:00:00') & (time <= '2021-12-31 23:00:00')]


test = data[(time >= '2022-01-01 00:00:00') & (time <= '2022-12-31 23:00:00')]




print(f'Train shape: {train.shape}')
print(f'Validation shape: {validation.shape}')
print(f'Test shape: {test.shape}')


In [ ]:
# Normalize the features
feature_scaler = StandardScaler()
train_features_scaled = feature_scaler.fit_transform(train.drop(columns=['time']))
validation_features_scaled = feature_scaler.transform(validation.drop(columns=['time']))
test_features_scaled = feature_scaler.transform(test.drop(columns=['time']))

# Normalize the target
target_scaler = StandardScaler()
train_target_scaled = target_scaler.fit_transform(train[['PM2.5']])
validation_target_scaled = target_scaler.transform(validation[['PM2.5']])
test_target_scaled = target_scaler.transform(test[['PM2.5']])

# Prepare the data for LSTM
def create_sequences(features, target, seq_length, pred_length):
    xs, ys = [], []
    for i in range(len(features) - seq_length - pred_length + 1):
        x = features[i:i + seq_length]
        y = target[i + seq_length:i + seq_length + pred_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 120
pred_length = 12

X_train, y_train = create_sequences(train_features_scaled, train_target_scaled, seq_length, pred_length)
X_val, y_val = create_sequences(validation_features_scaled, validation_target_scaled, seq_length, pred_length)
X_test, y_test = create_sequences(test_features_scaled, test_target_scaled, seq_length, pred_length)

# Reshape for LSTM [samples, time steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], X_val.shape[2]))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2]))


print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')
print(f'X_val shape: {X_val.shape}, y_val shape: {y_val.shape}')

In [ ]:
# Model Definition
model = Sequential([
    Input(shape=(seq_length, X_train.shape[2])),
    LSTM(300, activation='softsign', return_sequences=True),
    LSTM(300, activation='softsign', return_sequences=False),
    Dropout(0.5),
    Dense(pred_length, kernel_regularizer=tf.keras.regularizers.l2(0.03))
])

# Compile the model with MAE as an additional metric
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae'])

print(model.summary())


In [ ]:
# Early stopping and learning rate reduction callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, min_lr=1e-10)

# save running time
import time
start = time.time()

# Training the Model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

# Making Predictions
y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)
y_pred_test = model.predict(X_test)

end = time.time()
running_time = end - start
print(f'Running time: {running_time}')

In [ ]:
#saving predictions to npy files
np.save('y_pred_train_'+str(seq_length)+'_'+str(pred_length)+'.npy', y_pred_train)
np.save('y_pred_val_'+str(seq_length)+'_'+str(pred_length)+'.npy', y_pred_val)
np.save('y_pred_test_'+str(seq_length)+'_'+str(pred_length)+'.npy', y_pred_test)
np.save('y_train_'+str(seq_length)+'_'+str(pred_length)+'.npy', y_train)
np.save('y_val_'+str(seq_length)+'_'+str(pred_length)+'.npy', y_val)
np.save('y_test_'+str(seq_length)+'_'+str(pred_length)+'.npy', y_test)